# Análise da Produção de Energia - Dados ENEVA
por Diego Prado. Março, 2024.

## 1) Introdução

Este relatório tem como objetivo apresentar os procedimentos e resultados da exploração e análise dos dados disponibilizados pela companhia ENEVA e pelo Operador Nacional do Sistema Elétrico (ONS) referentes à produção de energia de usinas específicas do setor. Os dados considerados para este relatório referem-se ao período de 2016 à 2024, e estão disponíveis publicamente nos respectivos _sites_ de ambas as fontes (https://dados.ons.org.br/dataset/geracao-usina-2 e https://ri.eneva.com.br/informacoes-financeiras/planilhas-interativas/). 

Ao todo, onze usinas foram consideradas nesta pesquisa: Maranhão III, Maranhão IV, Maranhão V, Nova Venécia 2, Parnaíba IV, Parnaíba V, Jaguatirica II, Fortaleza, Porto de Sergipe I, Porto do Itaqui e Porto do Pecém II.

As etapas desenvolvidas ao longo desta atividade foram:

1. o desenvolvimento de procedimento automatizado, em Python, para aquisição, filtragem e tratamento dos dados de cada fonte, resultando em arquivos reduzidos em _.csv_;

2. o planejamento da estrutura e criação do banco de dados utilizando SQLite e DBBrowser;

3. o desenvolvimento de procedimento automatizado, em Python, para alimentação do banco de dados a partir do arquivos _.csv_;

4. a manipulação dos dados e criação de painel de análise dos mesmos, utilizando Power Bi;

5. a análise dos dados, apontadas neste relatório e

6. a diponibilização do Jupyter Notebook (_.ipynb_) com as linhas de código das etapas 1 e 3 e de seus respectivos produtos (_.csv_), do banco de dados (_.db_), do painel de análise (_.pbix_) e deste relatório (_.html_) no repositório git para compartilhamento.



    


## 2) Metodologia

### 2.1) Dados: aquisição, filtragem e tratamento

Por serem volumosos e provenientes de fontes distintas, os dados disponibilizados exigiram o desenvolvimento de procedimentos ligeiramente diferentes para sua aquisição e tratamento. Os dados disponibilizados pela ENEVA estão em um único arquivo de formato _.xlsx_ (MS Excel) e informam a geração de energia em periodicidade trimestral, do primeiro trimestre de 2016 ao terceiro trimestre de 2023. Já os dados disponibilizados pela ONS estão em diferentes arquivos no formato _.csv_ e informam a geração de energia em periodicidade horária. Os dados de janeiro de 2022 à março de 2024 estão em arquivos organizados de forma mensal. Os dados anteriores à janeiro de 2022 estão organizados em arquivos anuais.

Ao todo foram realizados três procedimentos diferentes para aquisição e filtragem dos dados: um para os dados da ENEVA (provenientes de arquivo único, 2016-2024), com passagens manuais e conversão de formato de arquivo (_.xlsx_ para _.csv_); um para os dados da ONS de janeiro de 2022 à março de 2024 (provenientes de múltiplos arquivos mensais); e um para os dados da ONS de janeiro de 2016 à dezembro de 2021,  (provenientes de múltiplos arquivos anuais). Ambos os últimos procedimentos foram automatizados e desenvolvidos em Python.

Os procedimentos direcionados aos dados da ONS resultaram em arquivos _.csv_ com estrutura idêntica, que foram concatenados em um único arquivo, concentrando a produção mensal de 2016 à 2024 das 11 usinas (_adb_ONS_2016_2024_mensal.csv_). Já o procedimento direcionado aos dados da ENEVA resultou em um arquivo único _.csv_ (_dadosOperacionais.csv_). O resultado final são arquivos com dados filtrados e selecionados da geração de energia das onze usinas escolhidas para a análise ao longo do período 2016-2024.

### 2.2) Banco de dados: estrutura, criação e alimentação

O banco de dados foi criado utilizando-se SQLite3, DBBrowser via conexão ODBC.

Para realizar uma análise com granulação temporal apropriada ao longo do período 2016-2024, o banco de dados foi estruturado de forma a permitir o registro de meses, trimestres e anos. Cada uma dessas dimensões temporais foi relacionada a uma tabela permitindo a adaptação ao tipo de dado recebido (geração de energia mensal ou trimestral).

As onze usinas foram listadas em uma tabela específica ('cegs') e relacionadas a outra tabela ('grupos') que registra quatro grupos de análise: Parnaiba Consolidated (Nova Venécia 2, Maranhão IV, Maranhão V, Maranhão III, Parnaíba IV e Parnaíba V), Coal Consolidated (Porto do Itaqui), (Porto do Pecém II), Roraima (Jaguatirica II) e Third-part LNG (Fortaleza, Porto de Sergipe I).

Finalmente, três tabelas para registro de geração de energia foram criadas: uma para os dados mensais da ONS ('ons_producao_meses'), uma para os dados trimestrais da ONS ('ons_producao_quarters') e uma para os dados trimestrais da ENEVA (eneva_dados). Campos adicionais foram criados nesta última tabela para guardar todas as informações provenientes do relatório de Dados Operacionais da empresa para possíveis análises futuras. 

<p align = "center">
<img src="./imagens/bd_eneva_fsc.jpg" alt="Figura 1 - Representação da estrutura do banco de dados utilizado para esta atividade." width="800"/>
</p>

A alimentação das tabelas 'ons_producao_meses', 'ons_producao_quarters' e 'eneva_dados' foi realizada de maneira automatizada. O procedimento, desenvolvido em Python, era responsável por ler os arquivos _.csv_, identificar os valores respectivos às tabelas de geração de energia e alimentá-las via sucessivos acessos em _sql querries_. As tabelas 'ano', 'mes', 'cegs', 'grupos' e 'quarters' foram alimentadas manualmente via DBBrowser.

## 3) Análise dos Dados

### 3.1) Cálculo da correlação entre dados ENEVA vs. ONS

Os dados de geração de energia de ambas as fontes apresentam periodicidades distintas. Também há carência de valores em determinados períodos e usinas. Uma análise correlacional se fez necessária para que uma possível inferência de valores entre ambos pudesse ser realizada.

O cálculo da correlação entre os dados de ambas as fontes se constituiu em analisar a soma trimestral da produção de todas as usinas no período de 2016 a 2024, resultando em 33 montantes trimestrais para ambos os conjuntos de dados. O valor do coeficiente de correlação encontrado entre eles foi 0.88, representando, portanto, uma alta correlação positiva entre eles.

<p align = "center">
<img src="./imagens/correlação.png" alt="Figura 2 - Gráfico de dispersão da soma da geração de energia trimestral dos dados a ONS e ENEVA." width="400"/>
</p>

### 3.2) Cálculo da média histórica trimestral (Dados ONS)

A série temporal desde 2016 nos permitiu identificar uma sazonalidade da geração de energia pelos trimestres ao longo do ano. Ao analisar a média e respectivo desvio padrão da geração de energia de todas as usinas em trimestres de diferentes anos, pôde-se inferir indicativos de sazonalidade periódica anual.

O cálculo da média e do desvio padrão considerou a produção de todas as indústrias durante os quatro trimestres do período entre 2016 a 2023 e o primeiro trimestre de 2024. Obteve-se valores médios de geração de energia iguais a 1.772 ± 214 MWh, 1.602 ± 190 MWh, 3.382 ± 404 MWh e 3.549 ± 392 MWh representativos para o primeiro, segundo, terceito e quarto trimestre, respectivamente. Isso indica uma tendência de aumento considerável da geração de energia na segunda metade dos anos.

<p align = "center">
<img src="./imagens/media_trim.png" alt="Figura 3 - Média da geração de energia por trimestre. A série temporal considerou os dados de 2016 a 2024 da ONS" width="400"/>
</p>

### 3.3) Cálculo de crescimento de geração de energia em um ano (GWh/trimestre)

Uma vez identificada a tendência de se ter uma maior geração de energia no segundo semestre dos anos analisados, foi realizada uma inferência da taxa de crescimento de geração de energia ao longo de cada ano. Com essa informação, pode-se ter uma ideia histórica de quais foram os anos que tiveram maior variação da geração de energia. Os valores indicaram que 2016, 2020, 2022 e 2023 obtiveram crescimento menor que 500 GWh por trimestre, enquanto 2021 teve um grande crescimento de geração de energia nos últimos trimestres quando comparado ao respectivo início de ano.

O cálculo da taxa de crescimento se dá pelo coeficiente angular da reta calculada entre a geração média do primeiro trimestre e a maior geração média do ano (sendo ela referente ao terceiro ou ao quarto trimestre do mesmo ano).

<p align = "center">
<img src="./imagens/taxaCrescimento.png" alt="Figura 4 - Taxa de variação de geração de energia ao longo do ano para cada ano. A série temporal considerou os dados de 2016 a 2023 da ONS" width="400"/>
</p>

### 3.4) Projeção de geração de energia para os próximos trimestres

Com os dados de taxa de crescimento de geração de energia ao longo da série histórica, pôde-se inferir um crescimento médio representativo e aplicá-lo aos dados da ONS referentes ao primeiro trimestre de 2024 com o objetivo de se obter uma projeção da geração para os futuros trimestres. O procedimento constituiu-se em calcular as gerações dos trimestres futuros por regressão linear utilizando-se a taxa de crescimento e respectivo desvio padrão como coeficientes.
Os valores inferidos de geração de energia para o segundo, terceiro e quarto trimestres de 2024 são 2631.59 ± 463.11 GWh, 3490.31 ± 926.22 GWh e 4349.02 ± 1389.32 GWh, respectivamente.

<p align = "center">
<img src="./imagens/proj24.png" alt="Figura 5 - Projeção mínima, média e máxima da geração de energia para os próximos trimestres de 2024." width="400"/>
</p>


## 4) Conclusões e Considerações Finais

Este relatório descreveu os procedimentos e análises realizados em dados de geração de energia referentes ao período de janeiro de 2016 à março de 2024 e provenientes de duas fontes distintas: os disponibilizados pela ONS e pela ENEVA. Apesar das diferenças de formato e periodicidade, das ausências de dados em algumas usinas e períodos, e das diferentes unidades de medida, os dados apresentaram uma alta correlação (0.88) após tratamento. Tal valor indica que as análises realizadas sobre os dados da ONS possam ser propagadas, mesmo que qualitativamente, para a realidade de geração de energia da ENEVA.

O processo de coleta de dados automatizado nos _sites_ das fontes se mostrou demorado devido ao volume dos mesmos. Ainda que o processo salve arquivos _.csv_ de back-up ao longo de sua execução, recomenda-se uma conexão de internet estável para que não haja complicações.

O banco de dados desenvolvido para a tarefa se mostrou consistente com sua estrutura para os propósitos solicitados e dinamismo nas representações em MS PowerBI, dada a granulação temporal dos dados. Campos adicionais na tabela de dados da ENEVA foram mantidos para futuras práticas e análises que venham a ser propostas, ainda que não tenham sido utilizados neste exercício.

A partir da análise dos dados, detectou-se uma peridiocidade sazonal ao longo dos anos, com os primeiros dois trimestres apresentando menor geração de energia, e posteriormente um significativo aumento de geração nos dois últimos trimestres. O crescimento de geração de energia ao longo do ano foi calculado para todo o período e identificou uma taxa de aumento média de 858.7 ± 567.2 GWh / trimestre. 

Finalmente, pôde-se criar uma projeção de geração de energia a partir dos valores adquiridos no primeiro trimestre de 2024 pela ENEVA, inferindo uma produção de 2631.6 ± 463.1 GWh, 3490.31 ± 926.22 GWh e 4349.02 ± 1389.32 GWh para o segundo, terceiro e quarto trimestres deste ano.

#### Checklist das Guidelines propostas

<ul>
    <li><input type="checkbox" id="tarefa1" name="tarefa1" value="Tarefa 1"><label for="tarefa1"> 1) Download data from ONS and Eneva websites, you can do it manually.</label>
        <ul>
            <li><input type="checkbox" id="subtarefa21" name="subtarefa21" value="Subtarefa 2.1"><label for="subtarefa21"> (Bonus: Create an automated script to download the data from ONS)</label></li>            
        </ul>        
    <li><input type="checkbox" id="tarefa2" name="tarefa2" value="Tarefa 2"><label for="tarefa2"> 2) ONS data is hourly. Aggregate it into monthly and quarterly data, filtering the fields as in the provided ceg_dict. You should save the processed data into an SQLite database. You can design the schema you think is more suitable for the problem.</label>
    </li>
    <li><input type="checkbox" id="tarefa3" name="tarefa3" value="Tarefa 3"><label for="tarefa3"> 3) Aggregate and load Eneva production data in the SQLite, in a separate table.</label>
    </li>    
    <li><input type="checkbox" id="tarefa4" name="tarefa4" value="Tarefa 4"><label for="tarefa4"> 4) Create a Power BI (or other BI tool) dashboard with data from your SQLite database. It should contain:</label>
    </li>    
        <ul>
            <li><input type="checkbox" id="subtarefa41" name="subtarefa41" value="Subtarefa 4.1"><label for="subtarefa41"> a seasonal plot with monthly data from ONS</label></li>
            <li><input type="checkbox" id="subtarefa42" name="subtarefa42" value="Subtarefa 4.2"><label for="subtarefa42"> a quarterly plot with ONS vs Eneva Production</label></li>
            <li><input type="checkbox" id="subtarefa43" name="subtarefa43" value="Subtarefa 4.3"><label for="subtarefa43"> Bonus: Add more historical data (2016-2021 files are annual).</label></li>
        </ul>
    </li>
    <li><input type="checkbox" id="tarefa5" name="tarefa5" value="Tarefa 5"><label for="tarefa5"> 5) Generate a Jupiter notebook report in html with your analysis and conclusions on the problem, it should contain:</label>
    </li>    
         <ul>
            <li><input type="checkbox" id="subtarefa51" name="subtarefa51" value="Subtarefa 5.1"><label for="subtarefa51"> A brief description of the problem</label>
            <li><input type="checkbox" id="subtarefa52" name="subtarefa52" value="Subtarefa 5.2"><label for="subtarefa52"> the methodology used</label>
            <li><input type="checkbox" id="subtarefa53" name="subtarefa53" value="Subtarefa 5.3"><label for="subtarefa53"> Descriptive statistics</label>
            <li><input type="checkbox" id="subtarefa54" name="subtarefa54" value="Subtarefa 5.4"><label for="subtarefa54"> Trend analysis</label>
            <li><input type="checkbox" id="subtarefa55" name="subtarefa55" value="Subtarefa 5.5"><label for="subtarefa55"> What is the correlation of both production series?</label>
            <li><input type="checkbox" id="subtarefa56" name="subtarefa56" value="Subtarefa 5.6"><label for="subtarefa56"> Is it possible to infer the next quarter Eneva’s production (GWh) using the provided ONS data? What would be the error for each quarter?</label>
        <ul>
    </li>
</ul>
